In [ ]:
# =========================
# CELLA 1 – Import librerie
# =========================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from gower import gower_matrix
import gower
from ucimlrepo import fetch_ucirepo
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import AgglomerativeClustering
import scipy.cluster.hierarchy as sch

sns.set(style="whitegrid")


In [ ]:
# =========================
# CELLA 2 – Caricamento dataset
# =========================
# Scarica il dataset Heart Failure
heart = fetch_ucirepo(id=519)

# Separa feature e target
X = heart.data.features
y = heart.data.targets

# Unisci in un unico DataFrame
df = pd.concat([X, y], axis=1)

# Stampa TUTTO il dataset
print(df.to_string())



In [ ]:
# =========================
# CELLA 3 – Info dataset
# =========================
# Informazioni generali
df.info()

# Null values
df.isnull().sum()

# Dimensioni
df.shape

# Statistiche descrittive
df.describe()

# Variabili
print(heart.variables)


In [ ]:
# =========================
# CELLA 4 – Analisi esplorativa (EDA)
# =========================

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
import os

sns.set(style="white")  

# ====== CREA CARTELLA DI SALVATAGGIO ======
save_path = r"C:\Users\marco\Desktop\UNI\PRINCIPI E MODELLI DELLA PERCEZIONE\img"
os.makedirs(save_path, exist_ok=True)
print(f"Le immagini verranno salvate in: {save_path}")

# =========================
# OTTENGO I COLORI REALI USATI DA SEABORN
# =========================

fig, ax = plt.subplots()
tmp = sns.boxplot(
    data=df,
    x='death_event',
    y='age',        
    hue='death_event',
    palette=palette,
    dodge=False,
    ax=ax,
    notch=True
)

# Estraggo i colori dalla legenda automatica
handles, labels = ax.get_legend_handles_labels()
real_colors = [h.get_facecolor() for h in handles]

plt.close(fig)  


# =========================
# DISTRIBUZIONE TARGET
# =========================

counts = df['death_event'].value_counts(normalize=True).reindex([0, 1])

plt.figure(figsize=(5,4))
ax = counts.plot(
    kind='bar',
    edgecolor='none',
    color=[real_colors[0], real_colors[1]]

# Pulizia grafico
ax.grid(False)
for spine in ax.spines.values():
    spine.set_visible(False)

ax.set_xticklabels(["Vivo", "Morto"], rotation=0)
plt.xlabel("Stato")
plt.ylabel("Frequenza")
plt.title("Distribuzione del target (death_event)")
plt.tight_layout()
plt.savefig(os.path.join(save_path, "target_distribution.png"))
plt.show()

# =========================
# COUNT PLOT VARIABILI BINARIE
# =========================

binary_vars = ['anaemia', 'diabetes', 'high_blood_pressure', 'sex', 'smoking']
palette = {0: "#1f77b4", 1: "#ff7f0e"}

n_bin = len(binary_vars)
n_cols = 3
n_rows = (n_bin + n_cols - 1) // n_cols
fig, axes = plt.subplots(n_rows, n_cols, figsize=(n_cols*5, n_rows*4))
axes = axes.flatten()

for i, col in enumerate(binary_vars):
    ax = axes[i]
    prop = (
        df.groupby(col)['death_event']
        .value_counts(normalize=True)
        .rename("proportion")
        .reset_index()
    )
    sns.barplot(
        data=prop,
        x=col,
        y="proportion",
        hue="death_event",
        order=[0, 1],
        hue_order=[0, 1],
        palette=palette,
        ax=ax
    )
    ax.set_title(f"Proporzione death_event per {col}")
    ax.set_ylim(0, 1)
    ax.grid(False)
    ax.set_xticklabels(["No", "Sì"])
    # Rimuovo le etichette degli assi
    ax.set_xlabel("")
    ax.set_ylabel("")
    for spine in ax.spines.values():
        spine.set_visible(False)
    handles, labels = ax.get_legend_handles_labels()
    new_labels = ["Vivo" if int(l) == 0 else "Morto" for l in labels]
    ax.legend(handles, new_labels, title="death_event")

for j in range(i+1, len(axes)):
    fig.delaxes(axes[j])

plt.tight_layout()
plt.savefig(os.path.join(save_path, "countplot_binarie.png"), dpi=300)
plt.show()


# =========================
# Boxplot variabili integer per death_event
# =========================

integer_vars = ['age', 'creatinine_phosphokinase', 'ejection_fraction', 
                'serum_sodium', 'time']

n_int = len(integer_vars)
n_cols = 3
n_rows = (n_int + n_cols - 1) // n_cols
fig, axes = plt.subplots(n_rows, n_cols, figsize=(n_cols*5, n_rows*4))
axes = axes.flatten()

for i, col in enumerate(integer_vars):
    ax = axes[i]
    sns.boxplot(
        data=df,
        x='death_event',
        y=col,
        hue='death_event',
        palette=palette,
        dodge=False,
        ax=ax,
        notch=True,
        legend=False
    )
    ax.set_title(f"{col} vs death_event")
    ax.set_xticklabels(["Vivo", "Morto"])
    lower = df[col].quantile(0.05)
    upper = df[col].quantile(0.95)
    ax.set_ylim(lower, upper)
    ax.set_xlabel("")
    ax.set_ylabel("")
    for spine in ax.spines.values():
        spine.set_visible(False)
    handles = [plt.Line2D([0], [0], color=palette[k], lw=6) for k in [0, 1]]
    labels = ["Vivo", "Morto"]
    ax.legend(handles, labels, title="death_event")

for j in range(i+1, len(axes)):
    fig.delaxes(axes[j])

plt.tight_layout()
plt.savefig(os.path.join(save_path, "boxplot_integer_per_target.png"), dpi=300)
plt.show()



# =========================
# Heatmap correlazioni
# =========================

df_encoded = df.copy()
for col in df_encoded.select_dtypes(include='object').columns:
    df_encoded[col] = LabelEncoder().fit_transform(df_encoded[col])

corr_matrix = df_encoded.corr()
ordered_cols = corr_matrix['death_event'].abs().sort_values(ascending=False).index

plt.figure(figsize=(12, 8))
sns.heatmap(
    corr_matrix.loc[ordered_cols, ordered_cols],
    annot=True, cmap="coolwarm", fmt=".2f", square=False,
    cbar=False, linewidths=0, linecolor='white'
)
plt.title("Heatmap (colonne ordinate per correlazione |death_event|)")
plt.tight_layout()
plt.savefig(os.path.join(save_path, "heatmap_correlazioni.png"), dpi=300)
plt.show()


In [ ]:
# =========================
# CELLA 5 – Preprocessing (Gower)
# =========================
import os
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.cluster import AgglomerativeClustering
from sklearn.manifold import MDS
import scipy.cluster.hierarchy as sch
import gower

# ====== CREA CARTELLA DI SALVATAGGIO ======
save_path = r"C:\Users\marco\Desktop\UNI\PRINCIPI E MODELLI DELLA PERCEZIONE\img"
os.makedirs(save_path, exist_ok=True)
print(f"Le immagini verranno salvate in: {save_path}")

# Elimina righe con NaN
df_clean = df.dropna().copy()

# Separiamo feature e target
X_clean = df_clean.drop("death_event", axis=1)
y_clean = df_clean["death_event"]

# Rimuovi le colonne che non devono essere considerate per il clustering
cols_to_remove = ['sex', 'smoking', 'time']
X_clustering = X_clean.drop(cols_to_remove, axis=1)

# Identifica colonne numeriche e categoriche
num_cols = X_clustering.select_dtypes(include=['int64', 'float64']).columns
cat_cols = X_clustering.select_dtypes(include=['object', 'category']).columns

# Converte solo le numeriche in float
X_clustering[num_cols] = X_clustering[num_cols].astype(float)

# Calcolo matrice distanza Gower
gower_dist = gower.gower_matrix(X_clustering)


# =========================
# CELLA 6 – Agglomerative Clustering con Gower (2 cluster)
# =========================

cluster_model_2 = AgglomerativeClustering(
    n_clusters=2,
    metric='precomputed',
    linkage='complete'
)

clusters_2 = cluster_model_2.fit_predict(gower_dist)
df_clean['cluster_2'] = clusters_2


# =========================
# CELLA 6a – Visualizzazione cluster MDS (2 cluster)
# =========================

mds = MDS(n_components=2, dissimilarity='precomputed', random_state=42, n_init=4)
coords = mds.fit_transform(gower_dist)

mds_df = pd.DataFrame(coords, columns=['Dim1', 'Dim2'])
mds_df['cluster'] = df_clean['cluster_2']

plt.figure(figsize=(8,6))
sns.scatterplot(
    x='Dim1', y='Dim2', hue='cluster', palette='Set1',
    data=mds_df, s=70, edgecolor='white', linewidth=1.2
)
sns.despine(trim=True, left=True, bottom=True)
plt.grid(False)
plt.xticks([])
plt.yticks([])
plt.title("Cluster (2) nello spazio MDS")
plt.tight_layout()
plt.savefig(os.path.join(save_path, "cluster_2_mds.png"))
plt.show()


# =========================
# CELLA 6b – Dendrogramma
# =========================

linkage_matrix = sch.linkage(gower_dist, method='complete')

plt.figure(figsize=(12, 6))
sch.dendrogram(linkage_matrix, labels=df_clean.index, leaf_rotation=90, leaf_font_size=8, color_threshold=0)
plt.grid(False)
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.gca().spines['left'].set_visible(False)
plt.gca().spines['bottom'].set_visible(False)
plt.title("Dendrogramma (distanze Gower)")
plt.xlabel("Campioni")
plt.ylabel("Distanza")
plt.tight_layout()
plt.savefig(os.path.join(save_path, "dendrogramma_gower.png"))
plt.show()


# =========================
# CELLA 6c – Analisi Cluster (2 cluster)
# =========================

print("=== ANALISI 2 CLUSTER ===")
for cluster_num in range(2):
    cluster_data = df_clean[df_clean['cluster_2'] == cluster_num]

    print(f"\n--- CLUSTER {cluster_num} ---")
    print(f"Numero pazienti: {len(cluster_data)}")
    print(f"Maschi: {cluster_data['sex'].sum()} ({cluster_data['sex'].mean()*100:.1f}%)")
    print(f"Femmine: {len(cluster_data)-cluster_data['sex'].sum()} ({100-cluster_data['sex'].mean()*100:.1f}%)")
    print(f"Vivi: {len(cluster_data[cluster_data['death_event']==0])}")
    print(f"Morti: {len(cluster_data[cluster_data['death_event']==1])}")
    print(f"Diabetici: {cluster_data['diabetes'].sum()}")
    print(f"Fumatori: {cluster_data['smoking'].sum()}")


# =========================
# CELLA 7 – Agglomerative Clustering (3 cluster)
# =========================

cluster_model_3 = AgglomerativeClustering(
    n_clusters=3,
    metric='precomputed',
    linkage='complete'
)

clusters_3 = cluster_model_3.fit_predict(gower_dist)
df_clean['cluster_3'] = clusters_3

mds_df_3 = pd.DataFrame(coords, columns=['Dim1', 'Dim2'])
mds_df_3['cluster'] = df_clean['cluster_3']

plt.figure(figsize=(8,6))
sns.scatterplot(
    x='Dim1', y='Dim2', hue='cluster', palette='Set1',
    data=mds_df_3, s=70, edgecolor='white', linewidth=1.2
)
sns.despine(trim=True, left=True, bottom=True)
plt.grid(False)
plt.xticks([])
plt.yticks([])
plt.title("Cluster (3) nello spazio MDS")
plt.tight_layout()
plt.savefig(os.path.join(save_path, "cluster_3_mds.png"))
plt.show()


# =========================
# CELLA 7b – Analisi Cluster (3 cluster)
# =========================

print("\n=== ANALISI 3 CLUSTER ===")
for cluster_num in range(3):
    cluster_data = df_clean[df_clean['cluster_3'] == cluster_num]

    print(f"\n--- CLUSTER {cluster_num} ---")
    print(f"Numero pazienti: {len(cluster_data)}")
    print(f"Vivi: {len(cluster_data[cluster_data['death_event']==0])}")
    print(f"Morti: {len(cluster_data[cluster_data['death_event']==1])}")


# =========================
# CELLA 8 – Agglomerative Clustering (4 cluster)
# =========================

cluster_model_4 = AgglomerativeClustering(
    n_clusters=4,
    metric='precomputed',
    linkage='complete'
)

clusters_4 = cluster_model_4.fit_predict(gower_dist)
df_clean['cluster_4'] = clusters_4

mds_df_4 = pd.DataFrame(coords, columns=['Dim1', 'Dim2'])
mds_df_4['cluster'] = df_clean['cluster_4']

plt.figure(figsize=(8,6))
sns.scatterplot(
    x='Dim1', y='Dim2', hue='cluster', palette='Set1',
    data=mds_df_4, s=70, edgecolor='white', linewidth=1.2
)
sns.despine(trim=True, left=True, bottom=True)
plt.grid(False)
plt.xticks([])
plt.yticks([])
plt.title("Cluster (4) nello spazio MDS")
plt.tight_layout()
plt.savefig(os.path.join(save_path, "cluster_4_mds.png"))
plt.show()


# =========================
# CELLA 8b – Analisi Cluster (4 cluster)
# =========================

print("\n=== ANALISI 4 CLUSTER ===")
for cluster_num in range(4):
    cluster_data = df_clean[df_clean['cluster_4'] == cluster_num]

    print(f"\n--- CLUSTER {cluster_num} ---")
    print(f"Numero pazienti: {len(cluster_data)}")
    print(f"Vivi: {len(cluster_data[cluster_data['death_event']==0])}")
    print(f"Morti: {len(cluster_data[cluster_data['death_event']==1])}")


# =========================
# CELLA 9 – Confronto Sopravvivenza per Cluster
# =========================

print("\n\n=== CONFRONTO SOPRAVVIVENZA PER CLUSTER ===")

print("\n--- 2 CLUSTER ---")
survival_rates_2 = []
for cluster_num in range(2):
    cluster_data = df_clean[df_clean['cluster_2'] == cluster_num]
    survival_rate = (len(cluster_data[cluster_data['death_event']==0]) / len(cluster_data)) * 100
    survival_rates_2.append((cluster_num, survival_rate, len(cluster_data)))

survival_rates_2.sort(key=lambda x: x[1], reverse=True)
for c, r, n in survival_rates_2:
    print(f"Cluster {c}: {r:.1f}% sopravvissuti ({n} pazienti)")


print("\n--- 3 CLUSTER ---")
survival_rates_3 = []
for cluster_num in range(3):
    cluster_data = df_clean[df_clean['cluster_3']==cluster_num]
    survival_rate = (len(cluster_data[cluster_data['death_event']==0]) / len(cluster_data)) * 100
    survival_rates_3.append((cluster_num, survival_rate, len(cluster_data)))

survival_rates_3.sort(key=lambda x: x[1], reverse=True)
for c, r, n in survival_rates_3:
    print(f"Cluster {c}: {r:.1f}% sopravvissuti ({n} pazienti)")


print("\n--- 4 CLUSTER ---")
survival_rates_4 = []
for cluster_num in range(4):
    cluster_data = df_clean[df_clean['cluster_4']==cluster_num]
    survival_rate = (len(cluster_data[cluster_data['death_event']==0]) / len(cluster_data)) * 100
    survival_rates_4.append((cluster_num, survival_rate, len(cluster_data)))

survival_rates_4.sort(key=lambda x: x[1], reverse=True)
for c, r, n in survival_rates_4:
    print(f"Cluster {c}: {r:.1f}% sopravvissuti ({n} pazienti)")


In [ ]:
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import numpy as np
import os

SAVE_PATH = r"C:\Users\marco\Desktop\UNI\PRINCIPI E MODELLI DELLA PERCEZIONE\img"
os.makedirs(SAVE_PATH, exist_ok=True)


# =====================================================
# Funzione per descrivere i cluster e assegnare colori
# =====================================================
def describe_cluster_with_color(df, cluster_col):
    descriptions = {}
    colors = {}

    for cluster in sorted(df[cluster_col].unique()):
        subset = df[df[cluster_col] == cluster]
        desc = []

        r, g, b = 200, 200, 200  # base neutra

        anaemia_label = None
        diabetes_label = None

        # ANEMIA
        if 'anaemia' in df.columns:
            if subset['anaemia'].eq(1).all():
                desc.append("ANEMICI")
                anaemia_label = "ANEMICI"
                r = 255; g = 50; b = 50
            elif subset['anaemia'].eq(0).all():
                desc.append("NON ANEMICI")
                anaemia_label = "NON ANEMICI"
                r = 255; g = 150; b = 150

        # DIABETE
        if 'diabetes' in df.columns:
            if subset['diabetes'].eq(1).all():
                desc.append("DIABETICI")
                diabetes_label = "DIABETICI"
                r = 50; g = 50; b = 255
            elif subset['diabetes'].eq(0).all():
                desc.append("NON DIABETICI")
                diabetes_label = "NON DIABETICI"
                r = 150; g = 150; b = 255

        # COMBINAZIONI
        if anaemia_label and diabetes_label:
            if anaemia_label == "NON ANEMICI" and diabetes_label == "DIABETICI":
                r, g, b = 121, 134, 210
            elif anaemia_label == "NON ANEMICI" and diabetes_label == "NON DIABETICI":
                r, g, b = 205, 183, 210
            elif anaemia_label == "ANEMICI" and diabetes_label == "DIABETICI":
                r, g, b = 128, 76, 153
            elif anaemia_label == "ANEMICI" and diabetes_label == "NON DIABETICI":
                r, g, b = 212, 126, 153

        descriptions[cluster] = ", ".join(desc)
        colors[cluster] = f"rgb({int(r)},{int(g)},{int(b)})"

    return descriptions, colors


# =====================================================
# Funzione Sankey
# =====================================================
def create_sankey_plot_descriptive(df_clean):
    labels = []
    sources = []
    targets = []
    values = []
    node_colors = []

    desc_2, color_2 = describe_cluster_with_color(df_clean, 'cluster_2')
    desc_3, color_3 = describe_cluster_with_color(df_clean, 'cluster_3')
    desc_4, color_4 = describe_cluster_with_color(df_clean, 'cluster_4')

    # Etichette cluster 2C
    for i in range(2):
        labels.append(f"2C_{i} ({len(df_clean[df_clean['cluster_2'] == i])}) {desc_2[i]}")
        node_colors.append(color_2[i])

    # Etichette cluster 3C
    for i in range(3):
        labels.append(f"3C_{i} ({len(df_clean[df_clean['cluster_3'] == i])}) {desc_3[i]}")
        node_colors.append(color_3[i])

    # Etichette cluster 4C
    for i in range(4):
        labels.append(f"4C_{i} ({len(df_clean[df_clean['cluster_4'] == i])}) {desc_4[i]}")
        node_colors.append(color_4[i])

    # Risultati finali
    survived_count = len(df_clean[df_clean['death_event'] == 0])
    not_survived_count = len(df_clean[df_clean['death_event'] == 1])
    labels.extend([f"Sopravvissuti ({survived_count})", f"Non Sopravvissuti ({not_survived_count})"])
    node_colors.extend(["lightgreen", "grey"]) 


    # Connessioni
    for c2 in range(2):
        mask2 = df_clean['cluster_2'] == c2
        for c3 in range(3):
            mask3 = df_clean['cluster_3'] == c3
            count = len(df_clean[mask2 & mask3])
            if count > 0:
                sources.append(c2)
                targets.append(2 + c3)
                values.append(count)

    for c3 in range(3):
        mask3 = df_clean['cluster_3'] == c3
        for c4 in range(4):
            mask4 = df_clean['cluster_4'] == c4
            count = len(df_clean[mask3 & mask4])
            if count > 0:
                sources.append(2 + c3)
                targets.append(5 + c4)
                values.append(count)

    for c4 in range(4):
        mask4 = df_clean['cluster_4'] == c4
        surv = len(df_clean[mask4 & (df_clean['death_event'] == 0)])
        dead = len(df_clean[mask4 & (df_clean['death_event'] == 1)])

        if surv > 0:
            sources.append(5 + c4)
            targets.append(9)
            values.append(surv)

        if dead > 0:
            sources.append(5 + c4)
            targets.append(10)
            values.append(dead)

    link_colors = [node_colors[src] for src in sources]

    fig = go.Figure(data=[go.Sankey(
        node=dict(
            pad=15,
            thickness=20,
            line=dict(color="black", width=0.5),
            label=labels,
            color=node_colors
        ),
        link=dict(
            source=sources,
            target=targets,
            value=values,
            color=link_colors
        ))])

    fig.update_layout(
        title_text="Sankey Diagram - Evoluzione Cluster e Risultati Finali",
        font_size=12,
        width=1200,
        height=700
    )

    # 💾 SALVATAGGIO IN HTML (MODIFICA RICHIESTA)
    fig.write_html(f"{SAVE_PATH}\\sankey_clusters.html")

    return fig, sources, targets, values, labels


print("Creando Sankey plot…")
sankey_fig, sources, targets, values, labels = create_sankey_plot_descriptive(df_clean)
sankey_fig.show()


# =====================================================
# Diagramma di flusso (Matplotlib)
# =====================================================
def create_flow_diagram(df_clean):
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(20, 8))

    transitions_2_3 = []
    for c2 in range(2):
        mask2 = df_clean['cluster_2'] == c2
        total2 = len(df_clean[mask2])
        for c3 in range(3):
            mask3 = df_clean['cluster_3'] == c3
            count = len(df_clean[mask2 & mask3])
            if count > 0:
                transitions_2_3.append((c2, c3, count, count / total2 * 100))

    for t in transitions_2_3:
        c2, c3, count, perc = t
        ax1.barh(f"2C_{c2}→3C_{c3}", count, color="skyblue")
        ax1.text(count, f"2C_{c2}→3C_{c3}", f"{count} ({perc:.1f}%)", va="center")

    ax1.set_title("Transizioni da 2 a 3 Cluster")
    ax1.set_xlabel("Numero pazienti")

    transitions_3_4 = []
    for c3 in range(3):
        mask3 = df_clean['cluster_3'] == c3
        total3 = len(df_clean[mask3])
        for c4 in range(4):
            mask4 = df_clean['cluster_4'] == c4
            count = len(df_clean[mask3 & mask4])
            if count > 0:
                transitions_3_4.append((c3, c4, count, count / total3 * 100))

    for t in transitions_3_4:
        c3, c4, count, perc = t
        ax2.barh(f"3C_{c3}→4C_{c4}", count, color="lightcoral")
        ax2.text(count, f"3C_{c3}→4C_{c4}", f"{count} ({perc:.1f}%)", va="center")

    ax2.set_title("Transizioni da 3 a 4 Cluster")
    ax2.set_xlabel("Numero pazienti")

    survival_by_cluster = []
    for c4 in range(4):
        mask4 = df_clean['cluster_4'] == c4
        surv = len(df_clean[mask4 & (df_clean['death_event'] == 0)])
        dead = len(df_clean[mask4 & (df_clean['death_event'] == 1)])
        survival_by_cluster.append((c4, surv, dead))

    x = np.arange(4)
    s_counts = [x[1] for x in survival_by_cluster]
    d_counts = [x[2] for x in survival_by_cluster]

    ax3.bar(x - 0.2, s_counts, 0.4, label="Sopravvissuti", alpha=0.7)
    ax3.bar(x + 0.2, d_counts, 0.4, label="Non Sopravvissuti", alpha=0.7)

    ax3.set_title("Risultati Finali per 4 Cluster")
    ax3.set_xticks(x)
    ax3.set_xticklabels([f"4C_{i}" for i in range(4)])
    ax3.legend()

    plt.tight_layout()

    # 💾 SALVATAGGIO PLOT MATPLOTLIB
    plt.savefig(f"{SAVE_PATH}\\diagramma_flusso.png", dpi=300)

    plt.show()
    return transitions_2_3, transitions_3_4, survival_by_cluster


print("\nCreando diagramma di flusso alternativo…")
trans_2_3, trans_3_4, survival_results = create_flow_diagram(df_clean)


# =====================================================
# Distribuzione cluster
# =====================================================
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

cluster_counts_2 = df_clean['cluster_2'].value_counts().sort_index()
axes[0].bar(cluster_counts_2.index, cluster_counts_2.values)
axes[0].set_title("Distribuzione 2 cluster")

cluster_counts_3 = df_clean['cluster_3'].value_counts().sort_index()
axes[1].bar(cluster_counts_3.index, cluster_counts_3.values)
axes[1].set_title("Distribuzione 3 cluster")

cluster_counts_4 = df_clean['cluster_4'].value_counts().sort_index()
axes[2].bar(cluster_counts_4.index, cluster_counts_4.values)
axes[2].set_title("Distribuzione 4 cluster")

plt.tight_layout()

# 💾 SALVATAGGIO DISTRIBUZIONE
plt.savefig(f"{SAVE_PATH}\\distribuzione_cluster.png", dpi=300)

plt.show()


In [ ]:
# =========================
# ANALISI COMPATTA DELLE DIFFERENZE TRA CLUSTER
# =========================

print("=" * 80)
print("DIFFERENZE TRA CLUSTER - MEDIE DELLE FEATURE USATE PER IL CLUSTERING")
print("=" * 80)

# Feature usate per il clustering (escluse sex, smoking, time)
clustering_features = ['age', 'anaemia', 'creatinine_phosphokinase', 'diabetes', 
                      'ejection_fraction', 'high_blood_pressure', 'platelets', 
                      'serum_creatinine', 'serum_sodium']

# Funzione per analizzare i cluster
def analyze_clusters_compact(df, cluster_col, n_clusters):
    print(f"\n{'='*50}")
    print(f"CLUSTERING CON {n_clusters} CLUSTER")
    print(f"{'='*50}")
    
    results = {}
    
    for cluster_id in range(n_clusters):
        cluster_data = df[df[cluster_col] == cluster_id]
        results[cluster_id] = {}
        
        print(f"\n--- CLUSTER {cluster_id} ({len(cluster_data)} pazienti) ---")
        
        # Calcola medie per tutte le feature di clustering
        for feature in clustering_features:
            if feature in cluster_data.columns:
                mean_val = cluster_data[feature].mean()
                results[cluster_id][feature] = mean_val
                
                if feature in ['age', 'ejection_fraction', 'serum_creatinine', 'serum_sodium']:
                    print(f"{feature:25}: {mean_val:6.1f}")
                else:
                    print(f"{feature:25}: {mean_val:6.2f}")
        
        # Aggiungi death_event per confronto
        death_rate = cluster_data['death_event'].mean() * 100
        results[cluster_id]['death_rate'] = death_rate
        print(f"{'death_rate':25}: {death_rate:6.1f}%")

# Analizza per 2, 3 e 4 cluster
analyze_clusters_compact(df_clean, 'cluster_2', 2)
analyze_clusters_compact(df_clean, 'cluster_3', 3)
analyze_clusters_compact(df_clean, 'cluster_4', 4)

# =========================
# CONFRONTO DIRETTO DELLE DIFFERENZE CHIAVE
# =========================

print("\n" + "=" * 80)
print("CONFRONTO DIRETTO - DIFFERENZE PRINCIPALI")
print("=" * 80)

def print_key_differences(df, cluster_col, n_clusters):
    print(f"\n📊 DIFFERENZE CHIAVE - {n_clusters} CLUSTER:")
    print("-" * 60)
    
    key_features = ['age', 'ejection_fraction', 'serum_creatinine', 'serum_sodium', 'death_event']
    
    for feature in key_features:
        print(f"\n{feature.upper():20} ", end="")
        values = []
        for cluster_id in range(n_clusters):
            cluster_data = df[df[cluster_col] == cluster_id]
            if feature == 'death_event':
                val = cluster_data[feature].mean() * 100
                values.append(val)
                print(f"| C{cluster_id}: {val:5.1f}% ", end="")
            else:
                val = cluster_data[feature].mean()
                values.append(val)
                print(f"| C{cluster_id}: {val:5.1f} ", end="")
        
        # Calcola range (differenza tra max e min)
        if len(values) > 1:
            diff = max(values) - min(values)
            if feature == 'death_event':
                print(f" | Δ: {diff:4.1f}%")
            else:
                print(f" | Δ: {diff:4.1f}")

# Stampa confronti per tutti i numeri di cluster
print_key_differences(df_clean, 'cluster_2', 2)
print_key_differences(df_clean, 'cluster_3', 3)
print_key_differences(df_clean, 'cluster_4', 4)

# =========================
# CRITERI PRINCIPALI DI SEPARAZIONE
# =========================

print("\n" + "=" * 80)
print("CRITERI PRINCIPALI DI SEPARAZIONE DEI CLUSTER")
print("=" * 80)

def identify_separation_criteria(df, cluster_col, n_clusters):
    print(f"\n🔍 COME SI SEPARANO I {n_clusters} CLUSTER:")
    
    # Trova le feature con maggiore variazione tra cluster
    feature_variation = {}
    
    for feature in clustering_features + ['death_event']:
        values = []
        for cluster_id in range(n_clusters):
            cluster_data = df[df[cluster_col] == cluster_id]
            if feature == 'death_event':
                val = cluster_data[feature].mean() * 100
            else:
                val = cluster_data[feature].mean()
            values.append(val)
        
        # Calcola coefficiente di variazione
        if max(values) > 0:
            variation = (max(values) - min(values)) / np.mean(values) * 100
            feature_variation[feature] = variation
    
    # Ordina per variazione decrescente
    top_features = sorted(feature_variation.items(), key=lambda x: x[1], reverse=True)[:3]
    
    print("   Criteri principali:")
    for feature, variation in top_features:
        print(f"   • {feature:20} (variazione: {variation:.1f}%)")

# Identifica criteri per ogni configurazione
identify_separation_criteria(df_clean, 'cluster_2', 2)
identify_separation_criteria(df_clean, 'cluster_3', 3)
identify_separation_criteria(df_clean, 'cluster_4', 4)

In [ ]:
# =========================
# CELLA – XGBoost migliorato con ottimizzazione parametri e soglia ottimale
# =========================

from xgboost import XGBClassifier, plot_tree
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report, confusion_matrix, roc_curve
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
from sklearn.metrics import ConfusionMatrixDisplay

save_path = r"C:\Users\marco\Desktop\UNI\PRINCIPI E MODELLI DELLA PERCEZIONE\img"
os.makedirs(save_path, exist_ok=True)

# =========================
# Feature e target
# =========================
X = df.drop(columns=['death_event', 'time', 'sex', 'smoking'])
y = df['death_event']

# Train/test split 75% – 25%
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42, stratify=y
)

# ===============================
# RandomizedSearchCV per parametri XGBoost
# ===============================
param_grid = {
    'n_estimators': [200, 300, 400],
    'max_depth': [3, 4, 5, 6],
    'learning_rate': [0.01, 0.03, 0.05, 0.1],
    'subsample': [0.7, 0.8, 0.9, 1.0],
    'colsample_bytree': [0.7, 0.8, 0.9, 1.0]
}

xgb = XGBClassifier(eval_metric='logloss', use_label_encoder=False)
search = RandomizedSearchCV(
    estimator=xgb,
    param_distributions=param_grid,
    n_iter=20,
    scoring='roc_auc',
    cv=3,
    verbose=1,
    random_state=42,
    n_jobs=-1
)

search.fit(X_train, y_train)
best_model = search.best_estimator_

print("\nBest parameters:", search.best_params_)

# ===============================
# Predizioni e probabilità
# ===============================
y_pred_proba = best_model.predict_proba(X_test)[:, 1]

# Trova soglia ottimale dalla curva ROC
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)
roc_dist = tpr - fpr
best_idx = np.argmax(roc_dist)
best_threshold = 0.40 #thresholds[best_idx]
print(f"Soglia ottimale calcolata: {best_threshold:.2f}")

# Predizione finale usando soglia ottimale
from sklearn.metrics import roc_curve, auc
y_pred = (y_pred_proba >= best_threshold).astype(int)
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(8,6))
plt.plot(fpr, tpr, color='blue', lw=2, label=f'ROC curve (AUC = {roc_auc:.3f})')
plt.plot([0, 1], [0, 1], color='grey', lw=1, linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Curva ROC')
plt.legend(loc='lower right')
plt.grid(True)
plt.show()

# ===============================
# Report delle performance
# ===============================
accuracy = accuracy_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_proba)

print("\n========= REPORT =========")
print(f"Accuracy: {accuracy:.3f}")
print(f"ROC-AUC: {roc_auc:.3f}\n")
print("Classification Report:")
print(classification_report(y_test, y_pred))

# Calcolo matrice di confusione
cm = confusion_matrix(y_test, y_pred)

plt.figure(figsize=(6, 5))

# Display della matrice di confusione
disp = ConfusionMatrixDisplay(
    confusion_matrix=cm,
    display_labels=["Vivo", "Morto"]  # Classi reali
)
disp.plot(values_format='d', cmap='Blues', ax=plt.gca())

# Etichette chiare sugli assi
plt.xlabel("Predizione")      # asse X = predetto dal modello
plt.ylabel("Classe reale")    # asse Y = valori veri
plt.title("Matrice di Confusione – XGBoost")

plt.grid(False)
plt.savefig(os.path.join(save_path, "confusion_matrix.png"))
plt.show()




# ==========================================
# Risultati paziente per paziente
# ==========================================
print("\n========== RISULTATI PER PAZIENTE ==========\n")
X_test_reset = X_test.reset_index(drop=True)
y_test_reset = y_test.reset_index(drop=True)

for i in range(len(X_test_reset)):
    reale = y_test_reset[i]
    predetto = y_pred[i]
    proba = y_pred_proba[i]
    corretto = "✔ CORRETTO" if reale == predetto else "❌ SBAGLIATO"
    print(f"Paziente {i+1:02d} → Reale={reale}, Predetto={predetto}, Prob={proba:.2f} → {corretto}")

# ===============================
# Predizione su nuovo paziente
# ===============================
nuovo_paziente = pd.DataFrame({
    'age': [75],
    'anaemia': [0],
    'creatinine_phosphokinase': [600],
    'diabetes': [1],
    'ejection_fraction': [35],
    'high_blood_pressure': [1],
    'platelets': [250000],
    'serum_creatinine': [2.0],
    'serum_sodium': [135]
})

prob_morte = best_model.predict_proba(nuovo_paziente)[:, 1][0]
predizione_binaria = int(prob_morte >= best_threshold)

print("\n========= NUOVO PAZIENTE =========")
print(f"Probabilità di morte: {prob_morte:.2f}")
print(f"Predizione (0= sopravvive, 1= muore): {predizione_binaria}")

# ===============================
# Visualizzazione albero XGBoost
# ===============================
os.environ["PATH"] += r";C:\Program Files\Graphviz\bin"
plt.figure(figsize=(20, 10))
plot_tree(best_model, num_trees=32, rankdir='LR')  # primo albero
plt.savefig(os.path.join(save_path, "albero_XGBoost.png"), dpi=600)
plt.show()

# Cerca un albero che usa anaemia e diabetes
booster = best_model.get_booster()
for i, tree in enumerate(booster.get_dump()):
    if 'anaemia' in tree and 'diabetes' in tree:
        print(f"Tree {i} contiene anaemia e diabetes")

# ===============================
# Feature importance (grafico)
# ===============================

importance = best_model.feature_importances_
features = X.columns

plt.figure(figsize=(8, 6))
plt.barh(features, importance, color='skyblue')

# Rimuove bordi e spine
ax = plt.gca()
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['bottom'].set_visible(False)

# Rimuove griglie
ax.grid(False)

plt.xlabel("Importanza")
plt.title("Feature Importance – XGBoost")
plt.savefig(os.path.join(save_path, "heatmap_correlazioni.png"), dpi=300)
plt.show()
